In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import nltk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [13]:
from nltk.corpus import stopwords


##### Data load and cleaning

In [2]:
df = pd.read_csv('../Data/archive/NewsCategorizer.csv')

In [3]:
df.head()

,category,headline,links,short_description,keywords
0,WELLNESS,143 Miles in 35 Days: Lessons Learned,https://www.huffingtonpost.com/entry/running-l...,Resting is part of training. I've confirmed wh...,running-lessons
1,WELLNESS,Talking to Yourself: Crazy or Crazy Helpful?,https://www.huffingtonpost.com/entry/talking-t...,Think of talking to yourself as a tool to coac...,talking-to-yourself-crazy
2,WELLNESS,Crenezumab: Trial Will Gauge Whether Alzheimer...,https://www.huffingtonpost.com/entry/crenezuma...,The clock is ticking for the United States to ...,crenezumab-alzheimers-disease-drug
3,WELLNESS,"Oh, What a Difference She Made",https://www.huffingtonpost.com/entry/meaningfu...,"If you want to be busy, keep trying to be perf...",meaningful-life
4,WELLNESS,Green Superfoods,https://www.huffingtonpost.com/entry/green-sup...,"First, the bad news: Soda bread, corned beef a...",green-superfoods


In [4]:
len(df['category'].unique())

10

In [5]:
Y=df[['category']]

In [6]:
Y.value_counts()

category      
BUSINESS          5000
ENTERTAINMENT     5000
FOOD & DRINK      5000
PARENTING         5000
POLITICS          5000
SPORTS            5000
STYLE & BEAUTY    5000
TRAVEL            5000
WELLNESS          5000
WORLD NEWS        5000
dtype: int64

In [7]:
X=df.loc[:,['headline','short_description','keywords']]

In [8]:
X_headline=df[['headline']]
X_description=df[['short_description']]
X_keywords=df[['keywords']]

In [9]:
X_headline.head(1)

,headline
0,143 Miles in 35 Days: Lessons Learned


In [10]:
X_description.iloc[0]

short_description    Resting is part of training. I've confirmed wh...
Name: 0, dtype: object

In [11]:
X_keywords.iloc[100]

keywords    shift-perspective
Name: 100, dtype: object

#### use only headline to detect News Category

##### clean and preprocessing the text

In [27]:
import re
from nltk.stem import WordNetLemmatizer

In [45]:
def cleanText(corpus):
    finalCorpus=[]
    lemmatizer = WordNetLemmatizer()
    sWords=list(set(stopwords.words('english')))
    for sen in corpus:
        word=[]
        sen=re.sub(r'[^a-zA-Z\s]', "", sen)
        wordList=sen.split()
        for w in wordList:
            if w not in sWords:
                word.append(lemmatizer.lemmatize(w.lower()))
        finalCorpus.append(" ".join(word))
    return finalCorpus
                
        

In [46]:
cleanedHeadText=cleanText(list(X_headline['headline']))

In [47]:
len(cleanedHeadText)

50000

##### create the vector and deepLearning model

In [49]:
tokenizer=tf.keras.preprocessing.text.Tokenizer(oov_token="OOV")

In [50]:
tokenizer.fit_on_texts(cleanedHeadText)

In [51]:
cleanedHeadSequence=tokenizer.texts_to_sequences(cleanedHeadText)

In [57]:
cleanedHeadPadSequence=tf.keras.preprocessing.sequence.pad_sequences(cleanedHeadSequence,padding="post")

In [58]:
cleanedHeadPadSequence

array([[ 1467,    23,   256, ...,     0,     0,     0],
       [  821,   716,   749, ...,     0,     0,     0],
       [16473,  1126,    42, ...,     0,     0,     0],
       ...,
       [12301, 12302,  7458, ...,     0,     0,     0],
       [   27,  3067,  8096, ...,     0,     0,     0],
       [ 2169,  2694,   369, ...,     0,     0,     0]])

In [70]:
output=LabelEncoder().fit_transform(Y.values.flatten())

In [74]:
Y['category'].unique()

array(['WELLNESS', 'POLITICS', 'ENTERTAINMENT', 'TRAVEL',
       'STYLE & BEAUTY', 'PARENTING', 'FOOD & DRINK', 'WORLD NEWS',
       'BUSINESS', 'SPORTS'], dtype=object)

In [71]:
output[1]

8

In [72]:
train_data, test_data, train_labels, test_labels = train_test_split(cleanedHeadPadSequence,output, test_size=0.2, random_state=0)

In [76]:
vSize=len(tokenizer.word_index)+1

In [79]:
inputL=len(cleanedHeadPadSequence[0])

In [90]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vSize, output_dim=50, input_length=40),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [91]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [92]:
model.fit(train_data, train_labels, epochs=20, validation_data=(test_data,test_labels))

Epoch 1/20


ValueError: in user code:

    File "c:\Users\Keerththanan\MYPROGRAM\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Keerththanan\MYPROGRAM\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Keerththanan\MYPROGRAM\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Keerththanan\MYPROGRAM\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\Keerththanan\MYPROGRAM\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "c:\Users\Keerththanan\MYPROGRAM\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_o

In [89]:
train_data

array([[    2,    29,   185, ...,     0,     0,     0],
       [   67,  2180,     6, ...,     0,     0,     0],
       [ 2121,  1291,  1449, ...,     0,     0,     0],
       ...,
       [   27,  2535,   312, ...,     0,     0,     0],
       [ 3113, 15985,   572, ...,     0,     0,     0],
       [   80,   128,   246, ...,     0,     0,     0]])